# 🔗 Documentation Explainer: The Prompt Chaining Pattern

Welcome to the prompt chaining lab! Prompt chaining (or prompt decomposition) involves breaking a task down into a series of steps. Each step calls an LLM and passes the output on to the next step. As part of each step, you can add error checking steps, or make calls to tools, knowledge bases, or other systems.

This pattern works best when:
- Tasks can be cleanly broken into fixed subtasks
- Trading latency for higher accuracy makes sense
- Each step builds on the previous one

Perfect for breaking down complex OpenSearch documentation! 🚀

In [ ]:
import chromadb
import boto3
from chromadb.config import Settings

# Initialize Chroma client from our persisted store
chroma_client = chromadb.PersistentClient(path="../../data/chroma")

session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime')

print("✅ Client setup complete!")

# Setup Helper Functions
We'll set up some basic helper functions from our rag basics examples for doing rag, calling Bedrock, and the retrieval portion of our local ChromaDB from the setup step

**Note on chroma:** Because we're using the same collection name and set a persistant client, it'll automatically load the dataset from disk so we don't need to reindex every time. 

In [2]:
from pydantic import BaseModel
from typing import List, Dict
from chromadb.api.types import EmbeddingFunction
from typing import List, Dict, Any
from chromadb.utils.embedding_functions import AmazonBedrockEmbeddingFunction


class RetrievalResult(BaseModel):
    id: str
    document: str
    embedding: List[float]
    distance: float
    metadata: Dict = {}


# Example of a concrete implementation
class ChromaDBRetrievalClient:

    def __init__(self, chroma_client, collection_name: str, embedding_function: AmazonBedrockEmbeddingFunction):
        self.client = chroma_client
        self.collection_name = collection_name
        self.embedding_function = embedding_function

        # Create the collection
        self.collection = self._create_collection()

    def _create_collection(self):
        return self.client.get_or_create_collection(
            name=self.collection_name,
            embedding_function=self.embedding_function
        )

    def retrieve(self, query_text: str, n_results: int = 5) -> List[RetrievalResult]:
        # Query the collection
        results = self.collection.query(
            query_texts=[query_text],
            n_results=n_results,
            include=['embeddings', 'documents', 'metadatas', 'distances']
        )

        # Transform the results into RetrievalResult objects
        retrieval_results = []
        for i in range(len(results['ids'][0])):
            retrieval_results.append(RetrievalResult(
                id=results['ids'][0][i],
                document=results['documents'][0][i],
                embedding=results['embeddings'][0][i],
                distance=results['distances'][0][i],
                metadata=results['metadatas'][0][i] if results['metadatas'][0] else {}
            ))

        return retrieval_results
    
from chromadb.utils.embedding_functions import AmazonBedrockEmbeddingFunction

# Define some experiment variables
EMBEDDING_MODEL_ID: str = 'amazon.titan-embed-text-v2:0'
COLLECTION_NAME: str = 'opensearch-docs-rag'

# This is a handy function Chroma implemented for calling bedrock. Lets use it!
embedding_function = AmazonBedrockEmbeddingFunction(
    session=session,
    model_name=EMBEDDING_MODEL_ID
)

# Create our retrieval task. All retrieval tasks in this tutorial implement BaseRetrievalTask which has the method retrieve()
# If you'd like to extend this to a different retrieval configuration, all you have to do is create a class that that implements
# this abstract class and the rest is the same!
chroma_os_docs_collection: ChromaDBRetrievalClient = ChromaDBRetrievalClient(
    chroma_client = chroma_client, 
    collection_name = COLLECTION_NAME,
    embedding_function = embedding_function
)

In [ ]:
# Verify that it works. 
len(chroma_os_docs_collection.retrieve("What is OpenSearch?", n_results=1))

Next we'll create some helper functions and classes

## Prompt Template
This is very useful for reusability. You can create your prompts in code or use a prompt management tool like Bedrock Prompt management to keep track of prompts. Even using these tools, it's important to create your own abstraction layer / implementation of the prompt so you can pass around typed objects.

Below is a sample prompt template implementation

In [4]:
from pydantic import BaseModel, Field
from typing import Dict, Any, Optional


HAIKU_MODEL_ID = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
class BasePrompt(BaseModel):
    """
    A streamlined base class for creating prompts with system and user components.
    """
    system_prompt: str
    user_prompt: str    
    inputs: Dict[str, Any] = Field(default_factory=dict)
    model_id: str = HAIKU_MODEL_ID
    hyperparams: Dict[str, Any] = Field(default_factory=lambda: {
        "temperature": 0.5,
        "maxTokens": 1000
    })

    # Just format the prompt if inputs were provided during initialization
    def __init__(self, **data):
        super().__init__(**data)
        # Format prompts if inputs were provided during initialization
        if self.inputs:
            self.format()

    def format(self, inputs: Dict[str, Any] = None) -> None:
        """Format system_prompt and user_prompt with inputs."""

        # Override the inputs if provided.
        inputs_to_use = inputs if inputs else self.inputs

        try:
            self.system_prompt = self.system_prompt.format(**inputs_to_use)
            self.user_prompt = self.user_prompt.format(**inputs_to_use)
        except KeyError as e:
            raise KeyError(f'Missing input value: {e}')
        except Exception as e:
            raise Exception(f'Error formatting prompt: {e}')
    

    def to_bedrock_messages(self, conversation: Optional[List[Dict[str, Any]]] = None) -> List[Dict[str, Any]]:
        """Convert to Bedrock messages format."""
        messages = conversation.copy() if conversation else []
        messages.append({"role": "user", "content": [{"text": self.user_prompt}]})
        return messages
    
    def to_bedrock_system(self, guard_content: Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]]:
        """Convert to Bedrock system format."""
        system = [{"text": self.system_prompt}]
        if guard_content:
            system.append({"guard": guard_content})
        return system

### Helper Functions
Now lets create some helper functions for calling bedrock and doing RAG.

In [5]:
from typing import Type

def call_bedrock(prompt: BasePrompt) -> str:
    kwargs = {
        "modelId": prompt.model_id,
        "inferenceConfig": prompt.hyperparams,
        "messages": prompt.to_bedrock_messages(),
        "system": prompt.to_bedrock_system(),
    }

    # Call Bedrock
    converse_response: Dict[str, Any] = bedrock.converse(**kwargs)
    # Get the model's text response
    return converse_response['output']['message']['content'][0]['text']

# Helper function to call bedrock
def do_rag(user_input: str, rag_prompt: Type[BasePrompt]) -> str:
    # Retrieve the context from the vector store
    retrieval_results: List[RetrievalResult] = chroma_os_docs_collection.retrieve(user_input, n_results=2)
    # Format the context into a string
    context: str = "\n\n".join([result.document for result in retrieval_results])

    print("Retrieval done")
    # Create the RAG prompt
    inputs: Dict[str, Any] = {"question": user_input, "context": context}
    rag_prompt: BasePrompt = rag_prompt(inputs=inputs)
    # Call Bedrock with the RAG prompt

    print("Calling Bedrock")
    return call_bedrock(rag_prompt)

## 1. Creating Our Documentation Explainer

We'll build a workflow that:
1. Extracts key concepts from OpenSearch documentation
2. Explains those concepts in simpler terms
3. Provides practical implementation examples

First lets define our prompts using the BasePrompt class we created above

In [6]:
SYSTEM_PROMPT = """
You are a helpful assistant that can explain OpenSearch documentation in simple terms.
"""

# Define reusable prompt templates as constants
EXTRACT_CONCEPTS_PROMPT_TEMPLATE = """
Using the users query, extract and list the key concepts from this OpenSearch documentation:

<query>
{question}
</query>

<documentation>
{context}
</documentation>

Focus on core principles and important technical details.
Format as a bulleted list.
"""

SIMPLIFY_EXPLANATION_PROMPT_TEMPLATE = """
Explain these OpenSearch concepts in simple, clear terms:
{concepts}

Write as if explaining to someone new to OpenSearch.
Include analogies where helpful.
"""

GENERATE_EXAMPLES_PROMPT_TEMPLATE = """
Create practical examples for implementing these OpenSearch concepts:
Concepts: {concepts}
Explanation: {explanation}

Include:
- Code snippets where relevant
- Step-by-step instructions
- Common pitfalls to avoid
"""

FORMAT_OUTPUT_TEMPLATE = """
# OpenSearch Documentation Breakdown

## Key Concepts
{concepts}

## Simple Explanation
{explanation}

## Implementation Examples
{examples}
"""

class ExtractConceptPrompt(BasePrompt):
    system_prompt: str = SYSTEM_PROMPT
    user_prompt: str = EXTRACT_CONCEPTS_PROMPT_TEMPLATE

class SimplifyExplanationPrompt(BasePrompt):
    system_prompt: str = SYSTEM_PROMPT
    user_prompt: str = SIMPLIFY_EXPLANATION_PROMPT_TEMPLATE

class GenerateExamplesPrompt(BasePrompt):
    system_prompt: str = SYSTEM_PROMPT
    user_prompt: str = GENERATE_EXAMPLES_PROMPT_TEMPLATE

class FormatOutputPrompt(BasePrompt):
    system_prompt: str = SYSTEM_PROMPT
    user_prompt: str = FORMAT_OUTPUT_TEMPLATE

Next lets create our State dictionary. We're using Pydantic models here but will convert to a dictionary before passing to our LangGraph graph

In [7]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# Define the WorkflowState using TypedDict
class ExplainerState(TypedDict):
    # Input
    query: str = ""
    # Extracted concepts
    concepts: str = ""
    # Simplified explanation
    explanation: str = ""
    # Generated examples
    examples: str = ""
    # Final formatted output
    final_output: str = ""
    # Track completion status
    complete: bool = False

Next, lets create our nodes as functions passing in the state dict and outputting the state dict

In [8]:
def extract_concepts(state: ExplainerState) -> ExplainerState:
    """Extracts key concepts from documentation"""
    print("Starting concept extraction...")
    concepts: str = do_rag(state['query'], ExtractConceptPrompt)
    state['concepts'] = concepts
    print(f"Concepts extracted")
    return state

def simplify_explanation(state: ExplainerState) -> ExplainerState:
    """Explains concepts in simpler terms"""
    print("Starting simplification of explanation...")
    inputs: Dict[str, Any] = {"concepts": state['concepts']}
    prompt: BasePrompt = SimplifyExplanationPrompt(inputs=inputs)

    explanation: str = call_bedrock(prompt)
    state['explanation'] = explanation
    print(f"Simplified explanation")
    return state

def generate_examples(state: ExplainerState) -> ExplainerState:
    """Provides practical implementation examples"""
    print("Generating examples...")
    inputs: Dict[str, Any] = {"concepts": state['concepts'], "explanation": state['explanation']}
    prompt: BasePrompt = GenerateExamplesPrompt(inputs=inputs)

    examples: str = call_bedrock(prompt)
    state['examples'] = examples
    print(f"Examples generated")
    return state

def format_output(state: ExplainerState) -> ExplainerState:
    """Formats the final documentation breakdown"""
    print("Formatting final output...")
    inputs: Dict[str, Any] = {
        "concepts": state['concepts'], 
        "explanation": state['explanation'], 
        "examples": state['examples']
    }
    prompt: BasePrompt = FormatOutputPrompt(inputs=inputs)

    final_output: str = call_bedrock(prompt)
    state['final_output'] = final_output
    print("Final output formatted.")
    return state
    
def init_state(query: str) -> ExplainerState:
    """Initialize the search state with a query."""
    return ExplainerState(
        query=query,
        concepts= "",
        explanation= "",
        examples= "",
        final_output= "",
        complete= False
    )

Lastly, lets compile our graph

In [9]:
def create_prompt_chain_workflow() -> StateGraph:
    # This is kind of frustrating. The StateGraph takes in a dict
    # so you only find out if you have a typing error at runtime..
    workflow = StateGraph(ExplainerState)

     # Add nodes to the graph
    workflow.add_node("extract_concepts", extract_concepts)
    workflow.add_node("simplify_explanation", simplify_explanation)
    workflow.add_node("generate_examples", generate_examples)
    workflow.add_node("format_output", format_output)
    
    # Define the workflow edges. These are sequential.
    workflow.add_edge(START, "extract_concepts")
    workflow.add_edge("extract_concepts", "simplify_explanation")
    workflow.add_edge("simplify_explanation", "generate_examples")
    workflow.add_edge("generate_examples", "format_output")
    workflow.add_edge("format_output", END)
    
    # Compile and return the workflow
    return workflow.compile()

graph: StateGraph = create_prompt_chain_workflow()

In [ ]:
# Lets visualize the graph to get a sense of what we're about to run
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

## 2. Run our Prompt Chain

Let's test our documentation explainer with a question related to security.

In [ ]:
# Pose a question.
question: str = "Create you explain OpenSearch's security configuration?"

# Initialize the state.
state: ExplainerState = init_state(query=question)

# Invoke the graph.
results: ExplainerState = graph.invoke(state)

print("📚 OpenSearch Security Documentation Breakdown")
print(results['final_output'])

## 3. Benefits of the Prompt Chaining Pattern

Our documentation breakdown approach offers several advantages:

✅ Each step has a clear, focused purpose

✅ Better understanding through progressive refinement

✅ Easy to modify or extend the chain


However, it's very slow because it's making lots of sequential calls with large amounts of outputs. In the next few sections, lets see how we can use some additional patterns to speed the workflow up!

Next, we'll explore how to use the routing pattern to handle different types of OpenSearch questions! 🚀
